In [79]:
import sympy as sp
import random

import symqups
from symqups import s as CahillGlauberS
from symqups import *
from symqups._internal.multiprocessing import *
from symqups.objects.scalars import *
from symqups.objects.operators import *
from symqups.manipulations import *
from symqups.ordering import *
from symqups.quantization import *
from symqups.cg import *
from symqups.objects import *
from symqups._internal.cache import *
from symqups.star_product import *
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import get_random_poly
# from symqups.eom import LindbladMasterEquation
from typing import Sequence

# a = [alpha(i) for i in range(10)]
# aop = [annihilateOp(i) for i in range(10)]
# ad = [alphaD(i) for i in range(10)]
# adop = [createOp(i) for i in range(10)]
# x = sp.Symbol("x")
# qq = [q(i) for i in range(10)]
# pp = [p(i) for i in range(10)]
# CahillGlauberS.val = sp.Rational(1/2)
# # symqups.hbar.val = sp.Number(1)
# CahillGlauberS
a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
pi = pi.val
CahillGlauberS.val = sp.Symbol("s")
s = CahillGlauberS.val

##

from symqups.star_product import _symb2comm, _CommutatorSymbol


In [ ]:
def Bopp_aop(X):
    with sp.evaluate(False):
        return aop*X - (s+1)/2*spq.Commutator(aop, X)
def Bopp_adop(X):
    with sp.evaluate(False):
        return adop*X + (s-1)/2*spq.Commutator(adop, X)

F = sp.Function("F")(aop, adop)

sp.simplify(CGTransform(Bopp_aop(Bopp_adop(Bopp_aop(F))).doit()))

(2*s**4*\alpha_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) - s**4 + 2*s**3*\alpha_{}**2*\overline{\alpha}_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) - 2*s**3*\alpha_{}*\overline{\alpha}_{} + s**3*\alpha_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) - s**3 + 2*s**2*\alpha_{}**2*\overline{\alpha}_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) - 2*s**2*\alpha_{}*\overline{\alpha}_{} - 3*s**2*\alpha_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) + s**2 - 2*s*\alpha_{}**2*\overline{\alpha}_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) + 2*s*\alpha_{}*\overline{\alpha}_{} - s*\alpha_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) + s + 6*\alpha_{}**2*\overline{\alpha}_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})) + 2*\alpha_{}*\overline{\alpha}_{} + \alpha_{}*CGTransform(F(\hat{a}_{}, \hat{a}^{\dagger}_{})))/8

In [116]:
Bopp_aop(F)

(s/2 + 1/2)*[F(\hat{a}_{}, \hat{a}^{\dagger}_{}),\hat{a}_{}] + \hat{a}_{}*F(\hat{a}_{}, \hat{a}^{\dagger}_{})

-s*\hat{a}_{}*\hat{a}^{\dagger}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}, \alpha_{2}, \overline{\alpha}_{2})/2 + s*\hat{a}^{\dagger}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}, \alpha_{2}, \overline{\alpha}_{2})*\hat{a}_{}/2 + \hat{a}_{}*\hat{a}^{\dagger}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}, \alpha_{2}, \overline{\alpha}_{2})/2 + \hat{a}^{\dagger}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}, \alpha_{2}, \overline{\alpha}_{2})*\hat{a}_{}/2

In [32]:
CGTransform.definition

Eq(\mathcal{W}_s\left[\hat{A}\right], \mathrm{tr}\left(\hat{A}\Delta_s\left(\bm{\alpha}\right)\right), \quad \Delta_s\left(\bm{\alpha}\right) = \int_{\mathbb{R}^{2N}}\mathrm{d}\bm{\beta}\exp\left(-\frac{1+s}{1-s}\left|\bm{\beta}\right|^2\right)\exp\left(\bm{\beta}\cdot\bm{\hat{a}}^\dagger -\overline{\bm{\beta}}\cdot\hat{\bm{a}}\right)\exp\left(\bm{\alpha}\cdot\overline{\bm{\beta}} - \overline{\bm{\alpha}}\cdot\bm{\beta} \right))

In [4]:
expr2 = adop*W + (s-1)/2 *spq.Commutator(adop, W)
expr2

-(s/2 - 1/2)*[StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}),\hat{a}^{\dagger}_{}] + \hat{a}^{\dagger}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{})

In [5]:
CGTransform(expr2.doit())

\overline{\alpha}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{})

In [6]:
class _Placeholder(Base):
    def _get_symbol_name_and_assumptions(cls, *custom_args):
        return r"\mathbin{\cdot}", {"commutator" : False}

class Commutator(sp.Expr):
    def __new__(cls, left : sp.Expr, right :sp.Expr = None):
        if not(right):
            right = _Placeholder()
            return super().__new__(cls, left, right) if left.has(HilbertSpaceObject) else sp.Integer(0)
        
        if ((not(left.has(HilbertSpaceObject)) or not(right.has(HilbertSpaceObject)))
            or not(get_oper_sub(left) & get_oper_sub(right))):
            return sp.Integer(0)
        
    def _latex(self, printer):
        return r"\left[{%s}, {%s}\right]" % (sp.latex(self.args[0]), sp.latex(self.args[1]))
    
    def apply(self, right):
        if not(isinstance(self.args[1], _Placeholder)):
            raise TypeError("Right operand already specified.")
        return Commutator(self.args[0], right)
    
    def doit(self, **hints):
        l = self.args[0]
        r = self.args[1]
        
        if isinstance(r, _Placeholder):
            raise TypeError("Right operand not specified.")
        
        out = l*r - r*l
        return out.doit(**hints)
        
Commutator(adop, annihilateOp(2))

0